In [ ]:
# libary imports
import geopandas as gpd
import osmnx as ox
import folium
import shapely as shp
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points
import networkx as nx
import matplotlib as plt

# update osmnx settings
useful_tags_ways = ox.settings.useful_tags_way + ['cycleway'] + ['bicycle'] + ['motor_vehicle']
ox.config(use_cache=True, 
          log_console=True,
          useful_tags_way=useful_tags_ways
          )

In [2]:
## inputs 
# show map
map = True # use True or False 

# input location
place = "Liverpool, United Kingdom"

In [ ]:
## get networks, nodes and boundary
road_network = ox.graph_from_place(place, network_type= 'drive', simplify=True)
bikeable_network = ox.graph_from_place(place, network_type= 'all', simplify=True) # we use the 'all' tag to ensure paths where walking and cycling allowed are accessed
boundary = ox.geocode_to_gdf(place)

# get bike parking
bicycle_parking_nodes = ox.features.features_from_place(place, tags={"amenity": ["bicycle_parking"]})

# ensure all returned items are converted to a point feature by converting any polygons into centriods
bicycle_parking_nodes['geometry'] = bicycle_parking_nodes.geometry.apply(lambda geom: geom.centroid if geom.type != 'Point' else geom)

# save bikeable network to geodataframes
bikeable_network_gdf = ox.graph_to_gdfs(bikeable_network, nodes=False, edges=True)

# set crs
bicycle_parking_nodes = bicycle_parking_nodes.to_crs('epsg:3857')
bikeable_network_gdf = bikeable_network_gdf.to_crs('epsg:3857')

In [16]:
## Locate the nearest street edge to each cycle parking
# this is required to find places where cycling can be infered via the presence of bike parking (if you can lock your bike there, chances are you can cycle to it!)

# Create a new GeoDataFrame to store the nearest edge information
nearest_edges_gdf = bicycle_parking_nodes.copy()

# Create spatial index (this speeds up the geoprocessing)
bikeable_network_gdf_sindex = bikeable_network_gdf.sindex

# Function to find the nearest edge for each point with a maximum search radius of 5 meters
def find_nearest_edge(point):
    possible_matches_index = list(bikeable_network_gdf_sindex.intersection(point.buffer(5).bounds))
    possible_matches = bikeable_network_gdf.iloc[possible_matches_index]
    
    nearest_edge = None
    min_distance = float('inf')
    
    for edge_idx, edge_row in possible_matches.iterrows():
        edge_geometry = edge_row['geometry']
        nearest_point_on_edge, _ = nearest_points(point, edge_geometry)
        distance = point.distance(nearest_point_on_edge)
        
        if distance < min_distance and distance <= 1:
            nearest_edge = edge_row['geometry']
            min_distance = distance
            
    return nearest_edge

# Apply the function to each point in the bicycle_parking_nodes gdf
nearest_edges_gdf['nearest_edge_geometry'] = nearest_edges_gdf['geometry'].apply(find_nearest_edge)

# Drop unnecessary columns from the DataFrame
nearest_edges_gdf.drop(columns=['geometry'], inplace=True)

# copy and rename columns for join
bikeable_network_gdf['shared_column'] = bikeable_network_gdf['geometry'].copy() 
nearest_edges_gdf['shared_column'] = nearest_edges_gdf['nearest_edge_geometry'].copy()

# Perform join on shared column 
merged_gdf = bikeable_network_gdf.merge(nearest_edges_gdf, on='shared_column', how='right')

# drop null geometries to clean the dataframe
merged_gdf = merged_gdf[merged_gdf.geometry.notna()]

# set crs and drop null geometries to clean the dataframe
merged_gdf = merged_gdf.to_crs('EPSG:4326')
merged_gdf = merged_gdf[merged_gdf.geometry.notna()]
if "bicycle_x" in merged_gdf.columns: # reason unknown, but some places make the bicycle column get renamed
    merged_gdf.rename(columns={"bicycle_x": "bicycle"}, inplace=True)

# List of values to filter
highway_values_to_drop = ["trunk", "motorway", "motorway_link", "primary", "trunk_link", "secondary", "tertiary", '"unclassified","residential"', "unclassified", "residential"]
bicycle_values_to_drop = ["dismount", "no"]
merged_gdf.drop(merged_gdf[merged_gdf['highway'].isin(highway_values_to_drop)].index, inplace=True)
merged_gdf.drop(merged_gdf[merged_gdf['bicycle'].isin(bicycle_values_to_drop)].index, inplace=True)

In [17]:
## now it is time to join the edges with cycle parking near
# Loop through each row in the GeoDataFrame
for idx, row in merged_gdf.iterrows():
    osmid = row['osmid']  # Get the osmid value from the current row
    
    # Find the edge in bikeable_network with matching osmid
    for u, v, edge_data in bikeable_network.edges(data=True):
        if 'osmid' in edge_data and edge_data['osmid'] == osmid:
            network_osmid = edge_data['osmid']  # Get the osmid from bikeable_network

            # Get all edges connected to the source edge
            connected_edges = list(bikeable_network.edges(u, data=True))
            connected_edges.extend(bikeable_network.edges(v, data=True))
            
            # Filter out excluded highway types
            excluded_highways = ['primary', 'secondary', 'trunk', 'tertiary', 'motorway', 'residential',
                                 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link',
                                 'tertiary_link', 'bus_guideway', 'raceway', 'busway',
                                 'corridor', 'proposed', 'construction', 'footway', 'steps', 'service']
            
            # Filter out no-cycling areas
            excluded_bicycle_values = ['no', 'dismount']

            # Filter out roads with max speeds
            excluded_max_speeds = ['20 mph', ' 30 mph', '40 mph', '50 mph', '60 mph']

            # Get valid connected edges
            valid_connected_edges = []
            for edge in connected_edges:
                edge_data = edge[2]
                if 'bicycle' not in edge_data or edge_data['bicycle'] not in excluded_bicycle_values:
                    if 'highway' in edge_data and not any(tag in edge_data['highway'] for tag in excluded_highways):
                        if 'maxspeed' not in edge_data or edge_data['maxspeed'] not in excluded_max_speeds:
                            valid_connected_edges.append(edge)
            #print("Valid Connected Edges:", valid_connected_edges)
            
            # Update edge attributes in the graph for valid connected edges
            for edge in valid_connected_edges:
                edge_data = edge[2]
                edge_data['potential_cycling'] = 'TRUE'
            
            break  # Stop searching after finding a match


In [ ]:
## view edges which can be deemed as potential cycling edges
# Convert network graph to GeoDataFrame
potential_cycling_gdf = ox.graph_to_gdfs(bikeable_network, nodes=False, edges=True)

# Filter rows where 'potential_cycling' is 'TRUE'
potential_cycling_gdf = potential_cycling_gdf[potential_cycling_gdf['potential_cycling'] == 'TRUE']

potential_cycling_gdf.explore()


In [19]:
# split into nodes and edges
bike_network_edges = ox.utils_graph.graph_to_gdfs(bikeable_network, nodes=False, edges=True)
bike_network_nodes = ox.utils_graph.graph_to_gdfs(bikeable_network, nodes=True, edges=False)

In [20]:
# keep on the street edge types that show 'proper' bike infrastructure

if 'cycleway' in bike_network_edges.columns:
    bike_network_edges = bike_network_edges[(bike_network_edges.highway == 'cycleway') 
                | (bike_network_edges.highway == 'bridleway') # this tag may need removing
                | (bike_network_edges.cycleway == 'crossing')
                | (bike_network_edges.cycleway == 'track')
                | (bike_network_edges.cycleway == 'separate')
                | (bike_network_edges.cycleway == 'oppesite_track') # this tag may need removing
                | ((bike_network_edges.bicycle == 'yes') & (bike_network_edges.highway=="pedestrian")) # this tag may need removing
                | ((bike_network_edges.bicycle == 'yes') & (bike_network_edges.highway== 'footway')) # this tag may need removing
                | (bike_network_edges.bicycle == 'designated') # this tag may need removing 
                | (bike_network_edges.bicycle == 'permissive') # this tag may need removing
                | (bike_network_edges.bicycle == 'use_sidepath')
                | (bike_network_edges.motor_vehicle == 'no') & (bike_network_edges.highway== 'residential')
                | (bike_network_edges.motor_vehicle == 'no') & (bike_network_edges.highway== 'unclassifed')
                | (bike_network_edges.potential_cycling == 'TRUE') # experimental tag
                ]

# Places in the USA often lack the 'cycleway' and 'bicycle' tags

else:
    bike_network_edges = bike_network_edges[(bike_network_edges.highway == 'cycleway') 
                | (bike_network_edges.highway == 'bridleway') # this tag may need removing
                ]


In [21]:
# join nodes and edges back into a network

bike_network =  ox.utils_graph.graph_from_gdfs(bike_network_nodes, bike_network_edges) 
# remove isolated node

bike_network = ox.utils_graph.remove_isolated_nodes(bike_network)

In [22]:
# get bike parking
bicycle_parking_nodes = ox.features.features_from_place(place, tags={"amenity": ["bicycle_parking"]})

In [ ]:
# set up map
m = ox.folium.plot_graph_folium(road_network, popup_attribute = 'highway' , zoom = 1, fit_bounds= True, weight = 0.5, opacity = 0.5)
folium.features.Choropleth(boundary, fill_color = '#03E7FF', fill_opacity = 0.04, line_color='#50DBEA', line_opacity=0.6 ).add_to(m)
m = ox.folium.plot_graph_folium(bike_network, graph_map = m, popup_attribute = 'highway' , zoom = 1, fit_bounds= True,color = 'red', weight= 1)

# Plot the bicycle parking nodes on the map
for _, poi in bicycle_parking_nodes.iterrows():
    centroid = poi['geometry'].centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.Icon(color='red', icon='bicycle', prefix='fa'),
    ).add_to(m)

def show_map(map):
    if map == True:
        print("Plotting...")
        return m

# plot map
show_map(map)

In [24]:
# convert network items to geodataframes

bicycle_parking_nodes_gdf = bicycle_parking_nodes